In [1]:
source("metagenomicFunction.R")
source("analysis_f.R")
source("Krona.R")
oldw <- getOption("warn") 
options(warn = -1)

In [2]:
library("phyloseq")
library('ape')
library('metagenomeSeq')
 
otu <- "phyloseq/otu_table.tsv"
tax <- "phyloseq/taxonomy_sklearn.tsv"
map <- "Metadata/mapping.txt"
tree <- "phyloseq/merged_rooted_tree.nwk"

phy <- phy.import(otu = otu, tax = tax, map = map, tree = tree)
phy <- subset_samples(phy, sample != "12_2" & sample != "20")
sample_data(phy)$status <- factor(sample_data(phy)$status, levels = c("sano","necrosi","periapicale","endodonticamente","endo-perio"))
phy

Caricamento del pacchetto richiesto: Biobase

Caricamento del pacchetto richiesto: BiocGenerics


Caricamento pacchetto: 'BiocGenerics'


I seguenti oggetti sono mascherati da 'package:stats':

    IQR, mad, sd, var, xtabs


I seguenti oggetti sono mascherati da 'package:base':

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Welcome to Bioconductor

    Vignettes contain introductory material; view with
    'browseVignettes()'. To cite Bioconductor, see
    'citation("Biobase")', and for packages 'citation("pkgname")'.



Caricamento pacchetto: 'Biobase'


Il seguente oggetto è mascherato da 'package:phyloseq':

    sampleNames


Caricamento del

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 2870 taxa and 40 samples ]
sample_data() Sample Data:       [ 40 samples by 4 sample variables ]
tax_table()   Taxonomy Table:    [ 2870 taxa by 8 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 2870 tips and 2795 internal nodes ]

In [3]:
#removing 0 counts                               
threshold <- rowSums(as.data.frame(otu_table(phy)))
indexes <- threshold[threshold > 0]
phy <- take_taxa(phy,names(indexes))

#removing uncharacterized
print(paste("Prefiltering:",ntaxa(phy)))
phy <- subset_taxa(phy,!is.na(Phylum) & !Phylum %in% c(""," p__uncharacterized","Unknown"))

#removing non bacterial            
print(paste("Filtering no bacterial:",ntaxa(phy)))
phy <- subset_taxa(phy,!is.na(Family) & !Family %in% c(" f__Mitochondria"," f__Chloroplast"))
phy <- subset_taxa(phy,!Genus %in% c(" Mitochondria"," g__Chloroplast", " g__Saccharimonadales"))
print(paste("Filtering Chloroplast & Mitochondria:",ntaxa(phy)))

#removing singleton 
threshold <- rowSums(as.data.frame(otu_table(phy))) 
indexes <- threshold[threshold > 1]
phy <- take_taxa(phy,names(indexes))
print(paste("Filtering Singleton:",ntaxa(phy)))


[1] "Prefiltering: 2835"
[1] "Filtering no bacterial: 2602"
[1] "Filtering Chloroplast & Mitochondria: 2469"
[1] "Filtering Singleton: 2461"


In [3]:
##subtypes
phy.necro <- subset_samples(phy,  status=="sano" | status=="necrosi")
phy.necro <- subset_samples(phy.necro, sample %in% as.vector(subset(sample_data(phy.necro), status=="necrosi")$sample))
phy.necro

phy.peri <- subset_samples(phy,  status=="sano" | status=="periapicale")
phy.peri <- subset_samples(phy.peri, sample %in% as.vector(subset(sample_data(phy.peri), status=="periapicale")$sample))
phy.peri


phy.periVsnecrosi <- subset_samples(phy,  status=="periapicale" | status=="necrosi")
phy.periVsnecrosi

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 2870 taxa and 18 samples ]
sample_data() Sample Data:       [ 18 samples by 4 sample variables ]
tax_table()   Taxonomy Table:    [ 2870 taxa by 8 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 2870 tips and 2795 internal nodes ]

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 2870 taxa and 18 samples ]
sample_data() Sample Data:       [ 18 samples by 4 sample variables ]
tax_table()   Taxonomy Table:    [ 2870 taxa by 8 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 2870 tips and 2795 internal nodes ]

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 2870 taxa and 18 samples ]
sample_data() Sample Data:       [ 18 samples by 4 sample variables ]
tax_table()   Taxonomy Table:    [ 2870 taxa by 8 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 2870 tips and 2795 internal nodes ]

In [4]:
filter.phy <- function(phy_agg,threshold_taxa,threshold_sample){
    cat(red(paste("N° taxa before filtering:",ntaxa(phy_agg),"\n")))

    threshold <- rowSums(as.data.frame(otu_table(phy_agg))) / sum(sample_sums(phy_agg))*100
    indexes <- threshold[threshold >= threshold_taxa]
    phy.f <- take_taxa(phy_agg,names(indexes))
    cat(red(paste("N° taxa after RARE filtering :",ntaxa(phy.f),"\n")))
    # seleziona i taxa presenti in almeno  

    min_samples <- round(threshold_sample*nsamples(phy.f))
    cols <- rowSums(as.data.frame(otu_table(phy.f)) != 0)
    taxas <- names(cols[ cols >= min_samples])
    phy.f <- take_taxa(phy.f,taxas)
    phy.f <- subset_taxa(phy.f, Phylum!= 'Unknown')
    cat(red(paste("N° taxa after SAMPLE filtering :",ntaxa(phy.f),"\n")))
    
    return(phy.f)
    }

deseq2.test <- function(phy.f,form){
    
    Q <- phyloseq_to_deseq2(phy.f,form)
    geoMeans = apply(counts(Q),1,gm_mean)
    D <- estimateSizeFactors(Q, geoMeans = geoMeans)
    dds <- DESeq(D)                                                             
    normalized_count <- as.data.frame(round(counts(D,normalized=TRUE)))
    otu_table(phy.f) <- otu_table(normalized_count, taxa_are_rows=TRUE)
    colnames(normalized_count) <- gsub("\\.","-",colnames(normalized_count))
    otu_table(phy.f) <- otu_table(normalized_count, taxa_are_rows=TRUE)
    res <- as.data.frame(results(dds))
    res <- res[order(res$padj),]
    return(res)
    }

In [5]:
library(ggpubr)
library(crayon)
library(hash)
library(DESeq2)


#results <- hash() 
results.des <- hash()
for(ty in c("necro","peri","periVsnecrosi")){
    print(paste("Type:",ty))
    #results[[ty]] <- hash()
    results.des[[ty]] <- hash()
    if(ty=="necro"){
        phy.fun<-phy.necro
        }else if(ty=="periVsnecrosi"){
        phy.fun<-phy.periVsnecrosi
        }else{
        phy.fun<-phy.peri
        }
    for(level in c("Phylum","Family","Genus","Species")){
        print(paste("Level",level))
        phy_agg <- microbiome::aggregate_taxa(phy.fun, level)
        if(level=="Genus"){
            phy_agg <- subset_taxa(phy_agg, Genus!= 'Unknown')
            }
    #RARE taxa filtering
    print("Filtering")
    #Get count of phyla
    threshold_taxa <- 0.005
    threshold_sample <- 0.3

    phy.f <- filter.phy(phy_agg,threshold_taxa,threshold_sample)
    
    ####METAGENOMSEQ
    metaSeqObject <- newMRexperiment(counts = as.data.frame(as(otu_table(phy.f),'matrix')),phenoData = AnnotatedDataFrame(sample_data(phy.f)), featureData = AnnotatedDataFrame(as.data.frame(tax_table(phy.f)))) 
    metaSeqObject  <- cumNorm( metaSeqObject , p=cumNormStatFast(metaSeqObject))
        
    # Define the metadata categories:
    status <- as.factor(pData(metaSeqObject)$status)
    type <- as.factor(pData(metaSeqObject)$type)
    sample <- as.factor(pData(metaSeqObject)$sample)
    
    cat(red(paste("Normalization factor...\n")))
    # Define the normalisation factor
    norm.factor <- normFactors(metaSeqObject)
    norm.factor <- log2(norm.factor/median(norm.factor) + 1)
    cat(red(paste("Creating model...\n")))
    # Create the model
    settings <- zigControl(maxit = 10, verbose = TRUE)
    if(ty=="periVsnecrosi"){
        mod <- model.matrix(~status+norm.factor)
        form <- as.formula("~status")
        res <- deseq2.test(phy.f,form)
        }else{
            mod <- model.matrix(~status+sample+norm.factor)
            form <- as.formula("~sample+status")
            res <- deseq2.test(phy.f,form)
        }
    #fit <- fitZig(obj = metaSeqObject, mod = mod, useCSSoffset = FALSE, control = settings)
    #coefs <- MRcoefs(fit, coef = 2, group = 3, number = 40)
    #sigtab = subset(coefs,coefs$adjPvalues < 0.05)
    sigtab.des = subset(res,res$padj < 0.05)
    ##Deseq2
    #print(paste("Found metagen:",length(rownames(sigtab))))
    print(paste("Found deseq:",length(rownames(sigtab.des))))
    #results[[ty]][[level]] <- rownames(sigtab) 
    results.des[[ty]][[level]] <- row.names(sigtab.des)
    }
}

Caricamento del pacchetto richiesto: ggplot2


Caricamento pacchetto: 'ggpubr'


Il seguente oggetto è mascherato da 'package:ape':

    rotate



Caricamento pacchetto: 'crayon'


Il seguente oggetto è mascherato da 'package:ggplot2':

    %+%


hash-2.2.6.1 provided by Decision Patterns


Caricamento del pacchetto richiesto: S4Vectors

Caricamento del pacchetto richiesto: stats4


Caricamento pacchetto: 'S4Vectors'


I seguenti oggetti sono mascherati da 'package:hash':

    values, values<-


Il seguente oggetto è mascherato da 'package:tidyr':

    expand


I seguenti oggetti sono mascherati da 'package:Matrix':

    expand, unname


I seguenti oggetti sono mascherati da 'package:base':

    expand.grid, I, unname


Caricamento del pacchetto richiesto: IRanges


Caricamento pacchetto: 'IRanges'


Il seguente oggetto è mascherato da 'package:phyloseq':

    distance


Il seguente oggetto è mascherato da 'package:grDevices':

    windows


Caricamento del pacchetto richiesto: Genomic

[1] "Type: necro"
[1] "Level Phylum"
[1] "Filtering"
N° taxa before filtering: 15 
N° taxa after RARE filtering : 13 
N° taxa after SAMPLE filtering : 9 


Default value being used.



Normalization factor...
Creating model...


converting counts to integer mode

using pre-existing size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



...

[1] "Found deseq: 0"
[1] "Level Species"
[1] "Filtering"
N° taxa before filtering: 442 
N° taxa after RARE filtering : 233 
N° taxa after SAMPLE filtering : 90 


Default value being used.



Normalization factor...
Creating model...


converting counts to integer mode

using pre-existing size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

-- note: fitType='parametric', but the dispersion trend was not well captured by the
   function: y = a/x + b, and a local regression fit was automatically substituted.
   specify fitType='local' or 'mean' to avoid this message next time.

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 51 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



[1] "Found deseq: 0"


In [6]:
library(ggpubr)

phy.res <- CSS_norm(phy)
for(level in c("Phylum","Family","Genus","Species")){

    phy_agg <- microbiome::aggregate_taxa(phy.res, level)
    if(level=="Genus"){
        phy_agg <- subset_taxa(phy_agg, Genus!= 'Unknown')
        formula <- as.formula("Family~Genus")
    }else if(level=="Species"){
        phy_agg <- subset_taxa(phy_agg, Species!= 'Unknown')
        formula <- as.formula("Genus~Species")
        }else if(level=="Family"){
        formula <- as.formula("Phylum~Family")
        }else{
        formula <- as.formula("~Phylum")
        }
    filter <- outersect(results[["necro"]][[level]],results[['peri']][[level]])
    ph <- microbiome::transform(phy_agg, "log10")
    ph <- prune_taxa(filter,ph)
    ph <- subset_taxa(ph, level != "Unknown" )
    data <- psmelt(ph)

    #comparisons <- c("necrosi", "periapicale")
    p <- ggboxplot(data, x = "status", y = "Abundance",  id = "sample", add = "jitter", repel ="TRUE" ,
     color = "status", line.color = "gray", line.size = 0.4,
     palette = "Set2",ylab='Abundance (log10)')+
    facet_wrap(formula) +
    rotate_x_text(45)+
      #stat_compare_means(comparisons = comparisons, label = "p.signif", hide.ns = TRUE, paired = TRUE, vjust = 1.5)+
    theme(plot.margin = unit(c(3,3,3,3), "lines"),axis.text=element_text(size=8),strip.text.x = element_text(size = 8),axis.title.x=element_blank(),
        axis.text.x=element_blank(),
        axis.ticks.x=element_blank())

    tiff(paste("risultati_sklearn/gruppi/DA_",level,"_status_NOintersecting.tiff"), height=20, width=30, units="cm", res=300)
print(p)
dev.off()

    }

Default value being used.



In [357]:
library(stringr)

if(startsWith(t,"k")){
    sp <- str_split(t,pattern = "__")[[1]][8]
    paste("s__",sp,sep="")
    }

[1] "s__oralis"

In [104]:
302.8 < 33.11111

[1] FALSE

In [109]:
library(dplyr)
library(stringr)
re <- results.des

var1 <- 'Sano Vs Periapicale'
var2 <- 'Sano Vs Necrosi'
var3 <- 'Periapicale Vs Necrosi'
for(level in c("Phylum","Family","Genus","Species")){
    phy_agg <- microbiome::aggregate_taxa(phy, level)
    summary <- psmelt(phy_agg) %>% group_by(status,psmelt(phy_agg)[,level]) %>% summarise(Mean = mean(Abundance),
                              .groups = 'drop')
    colnames(summary) <- c("status",level,"Mean")
    df <- data.frame()
    vect1 <- re[['peri']][[level]]
    vect2 <- re[['necro']][[level]]
    vect3 <- re[['periVsnecrosi']][[level]]
    taxa <- unique(c(vect1,vect2, vect3))
    for(el in taxa){
        s <- subset(summary,(summary[,level] ==el & status=="sano"))$Mean
        n <- subset(summary,(summary[,level] ==el & status=="necrosi"))$Mean
        p <- subset(summary,(summary[,level] ==el & status=="periapicale"))$Mean
        if(startsWith(el,"k")){
            sp <- str_split(el,pattern = "__")[[1]][8]
            el <- paste("s__",sp,sep="")
        }
        if(el %in% vect1){
            if(s>p){
                df[el,var1] <- 'down'
            }else{
                df[el,var1] <- 'up'
            }
        }else{
            df[el,var1] <- '-'
                    }
        if(el %in% vect2){
            if(s>n){
                df[el,var2] <- 'down'
            }else{
                df[el,var2] <- 'up'
            }
        }else{
            df[el,var2] <- '-'
        }
        if(el %in% vect3){
            if(p>n){
                df[el,var3] <- 'down'
            }else{
                df[el,var3] <-  'up'
            }
        }else{
            df[el,var3] <- '-'
        }
    }
    write.table(df,paste("risultati_sklearn/gruppi/deseq/",level,"confronti.txt"),sep="\t", quote=FALSE, row.names = TRUE)
    
}



In [ ]:
results

In [ ]:
library(ggpubr)

level <- "Genus"

phy_agg <- microbiome::aggregate_taxa(phy, level)
phy_agg <- subset_taxa(phy_agg, level!= 'Unknown')

filter <- 
ph <- microbiome::transform(phy_agg, "log10")
ph <- prune_taxa(filter,ph)
ph <- subset_taxa(ph, level != "Unknown" )
data <- psmelt(ph)

#comparisons <- c("necrosi", "periapicale")
p <- ggbarplot(data, x = "sample", y = "Abundance",  id = "sample",#, add = "jitter", repel ,
 color = "status", line.color = "gray", line.size = 0.4,
 palette = "Set2",ylab='Abundance (log10)')+
facet_grid(type~Phylum+Family+Genus) +
rotate_x_text(45)+
  #stat_compare_means(comparisons = comparisons, label = "p.signif", hide.ns = TRUE, paired = TRUE, vjust = 1.5)+
theme(plot.margin = unit(c(3,3,3,3), "lines"),axis.text=element_text(size=8),strip.text.x = element_text(size = 8))

tiff(paste("risultati_sklearn/DA_",level,"_status_selected.tiff"), height=20, width=30, units="cm", res=300)
print(p)
dev.off()

In [38]:
library(ggpubr)

for(level in c("Phylum","Family","Genus","Species")){
    for(type in c(names(results.des))){
    #filter <- unique(c(results.des[['necro']][[level]],results.des[['peri']][[level]],results.des[['periVsnecrosi']][[level]]))
    filter <- results.des[[type]][[level]]
    if(length(filter)!=0){
        phy_agg <- microbiome::aggregate_taxa(phy, level)
    phy_agg <- subset_taxa(phy_agg, level!= 'Unknown')

    ph <- microbiome::transform(phy_agg, "log10")
    ph <- prune_taxa(filter,ph)
    ph <- subset_taxa(ph, level != "Unknown" )
    data <- psmelt(ph)

if(type=="necro"){
    comparisons <- c("sano", "necrosi")
    }else if(type=="peri"){
    comparisons <- c("sano","periapicale")
    }else{
    comparisons <- c("necrosi","periapicale")
    }
p <- ggboxplot(data, x = "status", y = "Abundance",  id = "sample", add = "jitter", repel ="TRUE" ,
 color = "status", line.color = "gray", line.size = 0.4,
 palette = "Set2",ylab='Abundance (log10)')+
facet_wrap(as.formula(paste("~",level))) +
rotate_x_text(45)+
  #stat_compare_means(comparisons = comparisons, hide.ns = FALSE, paired = FALSE, vjust = 1.5)+
  #stat_compare_means(label.y = 3 ) 
theme(plot.margin = unit(c(3,3,3,3), "lines"),axis.text=element_text(size=8),strip.text.x = element_text(size = 8))

tiff(paste("risultati_sklearn/gruppi/deseq/DA_",level,"_",type,"_status_boxplot.tiff"), height=20, width=30, units="cm", res=300)
print(p)
dev.off()
    }
    }
}

In [133]:
level <- "Genus"
#filter <- c("g__Pseudoramibacter","g__Lautropia","g__Olsenella","g__Slackia","g__Mycoplasma","g__Enterococcus",
#"g__Mogibacterium","g__Peptostreptococcaceae_[XI][G-1]","g__Bacteroidetes_[G-5]","g__Tannerella","g__Dialister","g__Alloprevotella"
#,"g__Campylobacter","g__Peptostreptococcus","g__Bacteroidales_[G-2]","g__Lactobacillus","g__Paracoccus")

filter_green <-c("g__Olsenella","g__Pseudoramibacter","g__Slackia","g__Mogibacterium",
                 "g__Dialister","g__Peptostreptococcus")#,"g__Atopobium","g__Enterococcus","g__Pseudomonas","g__Peptostreptococcaceae_[XI][G-1]","g__Lachnospiraceae_[G-2]")
phy_agg <- microbiome::aggregate_taxa(phy.peri, level)

ph <- microbiome::transform(phy_agg, "log10")
ph <- prune_taxa(filter_green,ph)
ph <- subset_taxa(ph, level != "Unknown" )
data <- psmelt(ph)

comparisons <- c("sano", "periapicale")
p <- ggpaired(data, x = "status", y = "Abundance",  id = "sample", add = "jitter", repel ="TRUE" ,
 color = "status", line.color = "gray", line.size = 0.5,
 palette = "Set2",ylab='Abundance (log10)')+
facet_wrap(~Family+Genus) +
rotate_x_text(45)+
  stat_compare_means(comparisons = comparisons, label = "p.signif", hide.ns = TRUE, paired = TRUE, vjust = 1.5)+
theme(plot.margin = unit(c(3,3,3,3), "lines"),axis.text=element_text(size=8),strip.text.x = element_text(size = 8))

tiff(paste("risultati_sklearn/gruppi/deseq/DA_",level,"_status_boxplot_up_green.tiff"), height=20, width=30, units="cm", res=300)
print(p)
dev.off()

png 
  2

In [136]:
phy.status <- 
phy_agg <- microbiome::aggregate_taxa(phy, level)

ph <- microbiome::transform(phy_agg, "log10")
ph <- prune_taxa(filter_green,ph)
#ph <- subset_samples(ph, sample %in% c(subset(sample_data(ph),status=='periapicale')$sample))#,"necrosi"))
data <- psmelt(ph)
                     
p <- ggbarplot(data, x = "sample", y = "Abundance",  id = "sample",#, add = "jitter", repel ,
 color = "status", line.color = "gray", line.size = 0.4,
 palette = "Set2",ylab='Abundance (log10)')+
facet_grid(type~Phylum+Family+Genus) +
rotate_x_text(45)+
  #stat_compare_means(comparisons = comparisons, label = "p.signif", hide.ns = TRUE, paired = TRUE, vjust = 1.5)+
theme(plot.margin = unit(c(3,3,3,3), "lines"),axis.text=element_text(size=8),strip.text.x = element_text(size = 8))

tiff(paste("risultati_sklearn//gruppi/deseq/DA_",level,"_status_green.tiff"), height=20, width=30, units="cm", res=300)
print(p)
dev.off()

png 
  2